In [1]:
# using mean to build a baseline model, predict by the combination of all categorical variables

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from datetime import date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [4]:
# load
data = pd.read_csv('df_egm.csv', parse_dates=['order_date'])
# feature with no variance / duplicated feature
data.drop(['store_id', 'is_liquidation', 'is_cancelled'], axis = 1, inplace = True)
# duplicated
data = data[~data['id'].duplicated()]
# drop cancelled
data = data[data['quantity_cancelled'] == 0]

data['date'] = data['order_date'].dt.date
data['cost'] = data['cost_product'] + data['cost_shipping'] + data['cost_other']

In [5]:
ds_egm = data[['id', 'date','revenue_net', 'cost']].copy()
ds_egm['egm'] = 1 - ds_egm['cost']/ds_egm['revenue_net']
# exluding revenue_net = 0: can't calculate EGM
ds_egm.loc[ds_egm['revenue_net'] == 0, 'egm'] = np.nan

In [6]:
in_mask = ds_egm['egm'].abs()<=3
out_mask = ds_egm['egm'].abs()>3
nan_mask = ds_egm['egm'].isnull()
ntotal = ds_egm.shape[0]

In [7]:
data = data[in_mask]

In [8]:
cats = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id', 'carrier_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']

In [9]:
data['egm'] = 1 - data['cost']/data['revenue_net']

In [10]:
mean_egm = data.groupby(cats)['egm'].mean().copy()

In [11]:
ds_train = data[data['date'] < date(2018, 7, 17)].copy()
ds_test = data[data['date'] > date(2018, 7, 16)].copy()

In [12]:
train_egm = ds_train[['id', 'egm']].copy()
test_egm = ds_test[['id', 'egm']].copy()

In [13]:
# predict training set
train_pred = ds_train.set_index(cats).drop(['egm'], axis = 1).join(mean_egm)[['id', 'egm']].copy()

In [14]:
# filling missing carrier_id
cats_no_car = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']
fi_car = ds_train.groupby(cats_no_car)['egm'].mean().copy()
train_pred['egm'] = train_pred.reset_index().set_index(cats_no_car)['egm'].fillna(fi_car).values

In [15]:
# missing category_id
cats_no_cg = ['shipping_speed_id', 'supplier_id', 'class_id', 'carrier_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']
fi_cg = ds_train.groupby(cats_no_cg)['egm'].mean().copy()
train_pred['egm'] = train_pred.reset_index().set_index(cats_no_cg)['egm'].fillna(fi_cg).values

In [16]:
# missing category_id & carrier_id
cats_no_cgcar = ['shipping_speed_id', 'supplier_id', 'class_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']
fi_cgcar = ds_train.groupby(cats_no_cgcar)['egm'].mean().copy()
train_pred['egm'] = train_pred.reset_index().set_index(cats_no_cgcar)['egm'].fillna(fi_cgcar).values

In [17]:
# missing supplier_id & category_id
cats_no_spcg = ['shipping_speed_id', 'class_id', 'carrier_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']
fi_spcg = ds_train.groupby(cats_no_spcg)['egm'].mean().copy()
train_pred['egm'] = train_pred.reset_index().set_index(cats_no_spcg)['egm'].fillna(fi_spcg).values

In [18]:
# missing shipping_speed_id & carrier_id
cats_no_spcar = ['supplier_id', 'category_id', 'class_id', 'manufacturer_id', 'is_b2b', 'is_giftcard']
fi_spcar = ds_train.groupby(cats_no_spcar)['egm'].mean().copy()
train_pred['egm'] = train_pred.reset_index().set_index(cats_no_spcar)['egm'].fillna(fi_spcar).values

In [19]:
# calculate MAE
train_pred = train_pred.reset_index().drop(cats, axis = 1).set_index('id')
train_egm = train_egm.rename(index = str, columns = {'egm': 'egm_true'})
train_egm = train_egm.set_index('id').join(train_pred).rename(index = str, columns = {'egm': 'egm_pred'})
mean_absolute_error(train_egm['egm_true'], train_egm['egm_pred'])

0.10918519320598337

In [20]:
# predict testing set
test_pred = ds_test.set_index(cats).drop(['egm'], axis = 1).join(mean_egm)[['id', 'egm']].copy()

In [21]:
# filling missing carrier_id
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_car)['egm'].fillna(fi_car).values

In [22]:
# filling missing category_id
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_cg)['egm'].fillna(fi_cg).values

In [23]:
# missing category_id & carrier_id
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_cgcar)['egm'].fillna(fi_cgcar).values

In [24]:
# filling missing manufacturer_id & carrier_id
cats_no_mancar = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id', 'is_b2b', 'is_giftcard']
fi_mancar = ds_train.groupby(cats_no_mancar)['egm'].mean().copy()
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_mancar)['egm'].fillna(fi_mancar).values

In [25]:
# filling missing manufacturer_id & carrier_id & is_b2b
cats_no_mancarb = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id', 'is_giftcard']
fi_mancarb = ds_train.groupby(cats_no_mancarb)['egm'].mean().copy()
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_mancarb)['egm'].fillna(fi_mancarb).values

In [26]:
# filling missing manufacturer_id & carrier_id & is_giftcard & is_b2b
cats_no_mancargb = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id']
fi_mancargb = ds_train.groupby(cats_no_mancargb)['egm'].mean().copy()
test_pred['egm'] = test_pred.reset_index().set_index(cats_no_mancargb)['egm'].fillna(fi_mancargb).values

In [44]:
mis = test_pred[test_pred['egm'].isnull()].copy()

In [45]:
mis = mis.reset_index().drop('carrier_id', axis = 1)

In [46]:
ship = set(ds_train['shipping_speed_id'].unique())
sup = set(ds_train['supplier_id'].unique())
cat = set(ds_train['category_id'].unique())
cla = set(ds_train['class_id'].unique())
man = set(ds_train['manufacturer_id'].unique())
dcats = {'shipping_speed_id': ship, 'supplier_id': sup, 'category_id': cat, 'class_id': cla, 'manufacturer_id': man}

In [47]:
tofill = set()
c = {'shipping_speed_id': '1', 'supplier_id': '2', 'category_id': '3', 'class_id': '4', 'manufacturer_id': '5'}

In [48]:
for _, val in mis.iterrows():
    c1 = ['shipping_speed_id', 'supplier_id', 'category_id', 'class_id', 'manufacturer_id']
    c2 = ['1','2','3','4','5']
    for s in c1:
        if val[s] not in dcats[s]:
            c2.remove(c[s])
    tofill.add(''.join(c2))

In [49]:
tofill

{'1234', '12345', '13', '134', '1345', '2345'}

In [76]:
# filling missing
cats_fill = ['shipping_speed_id', 'category_id',  'is_b2b', 'is_giftcard']
fi = ds_train.groupby(cats_fill)['egm'].mean().copy()
test_pred['egm'] = test_pred.reset_index().set_index(cats_fill)['egm'].fillna(fi).values

In [77]:
test_pred.reset_index().isnull().sum()

shipping_speed_id        0
supplier_id              0
category_id            127
class_id                 0
carrier_id           15361
manufacturer_id          0
is_b2b                   0
is_giftcard              0
id                       0
egm                      0
dtype: int64

In [79]:
test_pred = test_pred.reset_index().drop(cats, axis = 1).set_index('id')

In [81]:
test_egm = test_egm.rename(index = str, columns = {'egm': 'egm_true'})

In [83]:
test_egm = test_egm.set_index('id').join(test_pred).rename(index = str, columns = {'egm': 'egm_pred'})

In [85]:
mean_absolute_error(test_egm['egm_true'], test_egm['egm_pred'])

0.11244553235762837